In [12]:
# 使用SimpleTagBased算法对Delicious2K数据进行推荐
# 原始数据集：https://grouplens.org/datasets/hetrec-2011/
# 数据格式：userID     bookmarkID     tagID     timestamp
import random
import math
import operator


class TagBased():
    # 构造函数
    def __init__(self, filename):
        self.filename = filename
        self.loadData()
        self.randomlySplitData(0.2)
        self.initStat()
        self.testRecommend()

    # 数据加载
    def loadData(self):
        print("开始数据加载...")
        filename = self.filename
        # 保存了用户对item的tag
        self.records = {}
        fi = open(filename)
        lineNum = 0
        for line in fi:
            lineNum += 1
            if lineNum == 1:
                continue
            uid, iid, tag, timestamp = line.split('\t')
            # 在数组中对应的下标-1
            uid = int(uid) - 1
            iid = int(iid) - 1
            tag = int(tag) - 1
            # record数据结构{user1:{item1:[tag1, tag2, tag3, tag4}, item3:[tag2, tag4, tag5]}, user2:{item2:[tag3, tag7, tag8]}...}
            self.records.setdefault(uid, {})
            self.records[uid].setdefault(iid, [])
            self.records[uid][iid].append(tag)
        fi.close()
        print("数据集大小为 %d." % (lineNum))
        print("设置tag的人数 %d." % (len(self.records)))
        print("数据加载完成\n")

    # 将数据集拆分为训练集和测试集
    # 可以改变seed数值来改变训练集和测试集的比例
    def randomlySplitData(self, ratio, seed=100):
        random.seed(seed)
        self.train = dict()
        self.test = dict()
        for u in self.records.keys():
            for i in self.records[u].keys():
                if random.random() < ratio:
                    self.test.setdefault(u, {})
                    self.test[u].setdefault(i, [])
                    for t in self.records[u][i]:
                        self.test[u][i].append(t)
                else:
                    self.train.setdefault(u, {})
                    self.train[u].setdefault(i, [])
                    for t in self.records[u][i]:
                        self.train[u][i].append(t)
        print("训练集样本数 %d, 测试集样本数 %d" % (len(self.train), len(self.test)))

    # 使用训练集，初始化user_tags, tag_items, user_items
    def initStat(self):
        records = self.train
        self.user_tags = dict()
        self.tag_items = dict()
        self.user_items = dict()
        self.tag_users = dict()
        self.item_users = dict()
        self.item_tags = dict()
        for u, items in records.items():
            for i, tags in items.items():
                for tag in tags:
                    # print tag
                    # 用户和tag的关系，数据结构{user1:{tag1: 3, tag3: 2, tag6: 4}, user2:{...}}
                    self._addValueToMat(self.user_tags, u, tag, 1)
                    # tag和item的关系，数据结构{tag1:{item1: 1,item2: 3, item4: 4}, tag2:{...}}
                    self._addValueToMat(self.tag_items, tag, i, 1)
                    # 用户和item的关系，数据结构{user1:{item1: 1,item2: 3, item4: 4}, user2:{...}}
                    self._addValueToMat(self.user_items, u, i, 1)
                    # tag和用户的关系，数据结构{tag1:{user1: 1,user2: 3, user4: 1}, tag2:{...}}
                    self._addValueToMat(self.tag_users, tag, u, 1)
                    # item和用户的关系，数据结构{item1:{user1: 1,user2: 3, user4: 4}, item2:{...}}
                    self._addValueToMat(self.item_users, i, u, 1)
                    # item和tag的关系，数据结构{item1:{tag1: 1,tag2: 3, tag4: 4}, item2:{...}}
                    self._addValueToMat(self.item_tags, i, tag, 1)
        print("user_tags, tag_items, user_items初始化完成.")
        print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d, tag_users大小 %d, item_users大小 %d, item_tags大小 %d" % (
        len(self.user_tags), len(self.tag_items), len(self.user_items), len(self.tag_users), len(self.item_users), len(self.item_tags)))

    # 设置矩阵 mat[index, item] = 1
    def _addValueToMat(self, mat, index, item, value=1):
        if index not in mat:
            mat.setdefault(index, {})
            mat[index].setdefault(item, value)
        else:
            if item not in mat[index]:
                mat[index][item] = value
            else:
                mat[index][item] += value

    # 使用测试集，计算准确率和召回率
    def precision_recall_F(self, N, alpha = 1):
        hit_simple_tag = 0
        hit_norm_tag = 0
        hit_TFIDF = 0
        h_recall = 0
        h_precision = 0
        for user, items in self.test.items():
            if user not in self.train:
                continue
            # 获取SimpleTag算法下Top-N推荐列表
            rank_simple_tag = self.recommend_simple_tag(user, N)
            for item, rui in rank_simple_tag:
                if item in items:
                    hit_simple_tag += 1
            # 获取NormTag算法下Top-N推荐列表
            rank_norm_tag = self.recommend_norm_tag(user, N)
            for item, rui in rank_norm_tag:
                if item in items:
                    hit_norm_tag += 1
            # 获取TFIDF算法下Top-N推荐列表
            rank_TFIDF = self.recommend_TFIDF(user, N)
            for item, rui in rank_TFIDF:
                if item in items:
                    hit_TFIDF += 1
            h_recall += len(items)
            h_precision += N
        # 计算各算法的精确值、召回率和F值（默认为F1）
        simple_tag_precision = hit_simple_tag / (h_precision * 1.0)
        simple_tag_recall = hit_simple_tag / (h_recall * 1.0)
        norm_tag_precision = hit_norm_tag / (h_precision * 1.0)
        norm_tag_recall = hit_norm_tag / (h_recall * 1.0)
        TFIDF_precision = hit_TFIDF / (h_precision * 1.0)
        TFIDF_recall = hit_TFIDF / (h_recall * 1.0)
        F_simple_tag = (alpha ** 2 + 1.0) * simple_tag_precision * simple_tag_recall / \
        (alpha ** 2 * (simple_tag_precision + simple_tag_recall))
        F_norm_tag = (alpha ** 2 + 1.0) * norm_tag_precision * norm_tag_recall / (alpha ** 2 * (norm_tag_precision + norm_tag_recall))
        F_TFIDF =  (alpha ** 2 + 1.0) * TFIDF_precision * TFIDF_recall / (alpha ** 2 * (TFIDF_precision + TFIDF_recall))
        # 返回所有算法准确率、召回率和F值（默认为1）
        return simple_tag_precision, simple_tag_recall, norm_tag_precision, norm_tag_recall, TFIDF_precision, TFIDF_recall, \
        F_simple_tag, F_norm_tag, F_TFIDF

    # 对用户user推荐Top-N item list（Simple-Tag Based算法）
    def recommend_simple_tag(self, user, N):
        recommend_items = dict()
        # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
        tagged_items = self.user_items[user]
        for tag, wut in self.user_tags[user].items():
            for item, wti in self.tag_items[tag].items():
                if item in tagged_items:
                    continue
                if item not in recommend_items:
                    recommend_items[item] = wut * wti
                else:
                    recommend_items[item] += wut * wti
        return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

    # 对用户user推荐Top-N item list（Norm-Tag Based算法）
    def recommend_norm_tag(self, user, N):
        recommend_items = dict()
        tagged_items = self.user_items[user]
        # 计算用户使用过的标签个数
        qty_wut = len(self.user_tags[user])
        for tag, wut in self.user_tags[user].items():
            # 计算某标签被多少个用户使用过
            qty_wtu = len(self.tag_users[tag])
            for item, wti in self.tag_items[tag].items():
                if item in tagged_items:
                    continue
                if item not in recommend_items:
                    recommend_items[item] = (wut * 1.0 / qty_wut) * (wti * 1.0 / qty_wtu)
                else:
                    recommend_items[item] += (wut * 1.0 / qty_wut) * (wti * 1.0 / qty_wtu)
        return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

    def recommend_TFIDF(self, user, N):
        recommend_items = dict()
        tagged_items = self.user_items[user]
        for tag, wut in self.user_tags[user].items():
            # 计算某标签被多少个用户使用过
            qty_wtu = len(self.tag_users[tag])
            for item, wti in self.tag_items[tag].items():
                # 计算某商品i被打过多少个标签
                qty_wit = len(self.item_tags[item])
                if item in tagged_items:
                    continue
                if item not in recommend_items:
                    recommend_items[item] = (wut * 1.0 / (math.log(1 + qty_wtu))) * (wti * 1.0 / (math.log(1 + qty_wit)))
                else:
                    recommend_items[item] += (wut * 1.0 / (math.log(1 + qty_wtu))) * (wti * 1.0 / (math.log(1 + qty_wit)))
        return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

    # 使用测试集，对推荐结果进行评估
    def testRecommend(self):
        print("推荐结果评估")
        print("%s %15s %15s %15s %14s %14s %12s %15s %15s %12s" % ('N', 'SimpleTagPre.', 'SimpleTagRe.', 'NormTagPre.', 
                                                                     'NormTagRe.', 'TFIDFPre.', 'TFIDFRe.', 'F_SimpleTag', 
                                                                     'F_NormTag', 'F_TFIDF'))
        for n in [5, 10, 20, 40, 60, 80, 100]:
            precision_simple_tag, recall_simple_tag, precision_norm_tag, recall_norm_tag, precision_TFIDF, recall_TFIDF, F_simple_tag, \
            F_norm_tag, F_TFIDF = self.precision_recall_F(n)
            print("%d %10.3f%% %14.3f%% %14.3f%% %14.3f%% %14.3f%% %12.3f%% %13.3f %16.3f %13.3f" % \
                  (n, precision_simple_tag * 100, recall_simple_tag * 100, precision_norm_tag * 100, recall_norm_tag * 100, 
                   precision_TFIDF * 100, recall_TFIDF * 100, F_simple_tag, F_norm_tag, F_TFIDF))


if __name__ == '__main__':
    stb = TagBased("C:/Users/Administrator/Desktop/RS/L2Data/user_taggedbookmarks-timestamps.dat")

开始数据加载...
数据集大小为 437594.
设置tag的人数 1867.
数据加载完成

训练集样本数 1860, 测试集样本数 1793
user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860, tag_users大小 36884, item_users大小 59555, item_tags大小 59555
推荐结果评估
N   SimpleTagPre.    SimpleTagRe.     NormTagPre.     NormTagRe.      TFIDFPre.     TFIDFRe.     F_SimpleTag       F_NormTag      F_TFIDF
5      0.829%          0.355%          0.907%          0.388%          0.896%        0.383%         0.005            0.005         0.005
10      0.633%          0.542%          0.638%          0.546%          0.610%        0.523%         0.006            0.006         0.006
20      0.512%          0.877%          0.507%          0.868%          0.445%        0.762%         0.006            0.006         0.006
40      0.381%          1.304%          0.356%          1.218%          0.342%        1.170%         0.006            0.006         0.005
60      0.318%          1.635%          0.287%          1.476%          0.280% 